In [1]:
import scanpy as sc
import matplotlib.pyplot as plt
import pandas as pd
import anndata as ad
import numpy as np
import sys
sys.path.append("..")
from rosenbaum import rosenbaum

found cupy installation, will try use the GPU to calculate the distance matrix.


In [2]:
adata = sc.datasets.ebi_expression_atlas(accession="E-MTAB-9221")

In [3]:
adata.obs["Factor Value[clinical history]"]

SAMEA6979313-AAACCCAAGACTCAAA            control
SAMEA6979313-AAAGAACCACCTGCTT            control
SAMEA6979313-AAAGGATGTCCCTCAT            control
SAMEA6979313-AAAGGGTGTCCCTCAT            control
SAMEA6979313-AAAGGTTGTCCCTCAT            control
                                      ...       
SAMEA6979322-TTTCCTCTCAACCTCC    severe COVID-19
SAMEA6979322-TTTCGATAGATTAGAC    severe COVID-19
SAMEA6979322-TTTGATCCAATAGTGA    severe COVID-19
SAMEA6979322-TTTGGTTAGAATTGTG    severe COVID-19
SAMEA6979322-TTTGGTTCATTGTGCA    severe COVID-19
Name: Factor Value[clinical history], Length: 6807, dtype: category
Categories (3, object): ['control', 'mild COVID-19', 'severe COVID-19']

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)  # Normalize each cell to have 10,000 total counts
sc.pp.log1p(adata)  # Logarithmize the data
sc.pp.highly_variable_genes(adata, n_top_genes=100)  # Select top 2000 variable genes
adata = adata[:, adata.var.highly_variable]  # Subset to variable genes
sc.pp.scale(adata, max_value=10)  # Scale each gene to unit variance and clip to max_value=10
sc.tl.pca(adata, svd_solver='arpack')
sc.pp.neighbors(adata, n_neighbors=30)  # Set n_neighbors and use significant PCs
sc.tl.umap(adata, min_dist=0.3)  # Set min_dist
sc.pl.umap(adata, color="Factor Value[clinical history]")  # Replace 'louvain' with desired cluster or metadata field

/data/bionets/je30bery/anaconda3/envs/scanpy/lib/python3.9/site-packages/scanpy/preprocessing/_scale.py:318: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
/data/bionets/je30bery/anaconda3/envs/scanpy/lib/python3.9/site-packages/scipy/sparse/_index.py:151: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [ ]:
rosenbaum(adata, group_by="Factor Value[clinical history]", test_group="mild COVID-19", reference="Control", metric="euclidean", rank=False)